# **Sentiment Analysis on COVID-19 related Tweets (Machine Learning: Countvectorizer)**


In [ ]:
# We have to mount
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
import base64
import numpy as np 
import pandas as pd


#Plotly imports
import plotly.offline as py
py.init_notebook_mode(connected=True)
import plotly.graph_objs as go
import plotly.tools as tls
import re
import nltk
import string



# Other imports
from collections import Counter
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.decomposition import NMF, LatentDirichletAllocation
from matplotlib import pyplot as plt
%matplotlib inline
%matplotlib notebook

# Reading CSV File


In [ ]:
#TextBlob
#train=pd.read_csv('/content/gdrive/MyDrive/Colab Notebooks/Text_Blob_PolarityAnalysis.csv')
#Original 45,007
#train=pd.read_csv('/content/gdrive/MyDrive/Colab Notebooks/COVID-19MasterFinalDataset.csv')
#36k Tweets
#train=pd.read_csv('/content/gdrive/MyDrive/Colab Notebooks/36kCleanedCOVID-19MasterFinalDataset.csv')
#36k Tweets
train=pd.read_csv('/content/gdrive/MyDrive/Colab Notebooks/SentimentAnalysisCOVID-19MasterFinalDataset.csv')
train.head(10)

,Date,Tweets,Translated,Polarity,Sentiment,length
0,Mar,Me to COVID-19: pic.twitter.com/OLX9LTjTsW,me to covid,0.000000,Neutral,11
1,May,So many realizations during ECQ because of COV...,so many realizations during ecq because of cov...,0.300000,Positive,153
2,Mar,Murag sure pko na magka covid kesa magbalik mi...,it's like a covid covid kesa back to cejay hah...,0.100000,Positive,51
3,Apr,"While we are all fighting against COVID-19, me...",while we are all fighting against covid meanwh...,0.000000,Neutral,112
4,Mar,"Taong bahay, dahil sa covid-19😴",person house because of covid,0.000000,Neutral,29
5,Mar,"mamamatay ata ako sa stress, hindi sa covid.",I will die in stress not in covid,0.000000,Neutral,33
6,Jun,I want to join kaso may pangamba pa rin ng COV...,I Want To Join Case There is still a covid,0.000000,Neutral,42
7,Mar,"So gi confiscate ang akuang alcohol na 75mL, u...",So confiscate the Akuang Alcohol ML then the w...,0.000000,Neutral,62
8,Apr,I choose you to be a positive from covid 19 ch...,i choose you to be a positive from covid char...,0.227273,Positive,53
9,Apr,"Jgh, then ligo. Grabe covid stop na. 🤷🏻‍♂️🙏",jgh then ligo grabe covid stop na 🤷,0.000000,Neutral,35


In [ ]:
train.shape

(44709, 6)

# **Choose Translated Tweet and its Polarity**



In [ ]:
new_data = train[['Translated','Sentiment']]
#new_data = train[['Translated','Analysis']]
new_data.head(10)

,Translated,Sentiment
0,me to covid,Neutral
1,so many realizations during ecq because of cov...,Positive
2,it's like a covid covid kesa back to cejay hah...,Positive
3,while we are all fighting against covid meanwh...,Neutral
4,person house because of covid,Neutral
5,I will die in stress not in covid,Neutral
6,I Want To Join Case There is still a covid,Neutral
7,So confiscate the Akuang Alcohol ML then the w...,Neutral
8,i choose you to be a positive from covid char...,Positive
9,jgh then ligo grabe covid stop na 🤷,Neutral


In [ ]:
def convert_label(sentiment):
 
  for i in range(len(new_data)):
    if sentiment == "Negative":
      return -1;
    elif sentiment == "Neutral":
      return 0;
    elif sentiment == "Positive":
      return 1;

new_data["Analysis"] = new_data["Sentiment"].apply(convert_label)  
new_data.head()


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:11: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



,Translated,Sentiment,Analysis
0,me to covid,Neutral,0
1,so many realizations during ecq because of cov...,Positive,1
2,it's like a covid covid kesa back to cejay hah...,Positive,1
3,while we are all fighting against covid meanwh...,Neutral,0
4,person house because of covid,Neutral,0


In [ ]:
X = new_data['Translated'] 
y = new_data['Analysis']

In [ ]:
new_data.shape

(44709, 3)

# **Data Cleaning**

# **Data Definition**


In [ ]:
print('Dataset size:',new_data.shape)
print('Columns are:',new_data.columns)
new_data.info()

Dataset size: (44709, 3)
Columns are: Index(['Translated', 'Sentiment', 'Analysis'], dtype='object')
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 44709 entries, 0 to 44708
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Translated  44709 non-null  object
 1   Sentiment   44709 non-null  object
 2   Analysis    44709 non-null  int64 
dtypes: int64(1), object(2)
memory usage: 1.0+ MB


#**Train Test Split**

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score  
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=0)

# **Countvectorizer**

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

count_vect = CountVectorizer()
count_vect_train = count_vect.fit_transform(X_train)
count_vect_test = count_vect.transform(X_test)

# **Random Forest**

In [ ]:
from sklearn.ensemble import RandomForestClassifier

random_forest = RandomForestClassifier(n_estimators=50, random_state=50)  
random_forest.fit(count_vect_train, y_train)

predictions_randomforest = random_forest.predict(count_vect_test)
print(confusion_matrix(y_test,predictions_randomforest ))
print(classification_report(y_test,predictions_randomforest))
print('Accuracy Score: ',accuracy_score(y_test, predictions_randomforest))
print('Train Accuracy Score: ',random_forest.score(count_vect_train,y_train))
print('Test Accuracy Score: ',random_forest.score(count_vect_test,y_test))

[[ 932  262  373]
 [   7 3407  146]
 [  38  229 3548]]
              precision    recall  f1-score   support

          -1       0.95      0.59      0.73      1567
           0       0.87      0.96      0.91      3560
           1       0.87      0.93      0.90      3815

    accuracy                           0.88      8942
   macro avg       0.90      0.83      0.85      8942
weighted avg       0.89      0.88      0.88      8942

Accuracy Score:  0.8820174457615746
Train Accuracy Score:  0.9999440825341795
Test Accuracy Score:  0.8820174457615746


# **Multinomial Naive Bayes**

In [ ]:
from sklearn.naive_bayes import MultinomialNB

mnb= MultinomialNB(alpha = 0.003).fit(count_vect_train, y_train)

predictions_mnb = mnb.predict(count_vect_test)

print(confusion_matrix(y_test,predictions_mnb))  
print(classification_report(y_test,predictions_mnb))  
print('Accuracy Score: ',accuracy_score(y_test, predictions_mnb))
print('Train Accuracy Score: ',mnb.score(count_vect_train,y_train))
print('Test Accuracy Score: ',mnb.score(count_vect_test,y_test))


[[1034   93  440]
 [ 226 2679  655]
 [ 302  154 3359]]
              precision    recall  f1-score   support

          -1       0.66      0.66      0.66      1567
           0       0.92      0.75      0.83      3560
           1       0.75      0.88      0.81      3815

    accuracy                           0.79      8942
   macro avg       0.78      0.76      0.77      8942
weighted avg       0.80      0.79      0.79      8942

Accuracy Score:  0.7908745247148289
Train Accuracy Score:  0.95143568093494
Test Accuracy Score:  0.7908745247148289


# **Support Vector Machine**

In [ ]:
from sklearn.svm import LinearSVC 

linSVC = LinearSVC(C= 1.0, loss= 'squared_hinge', max_iter= 10000, penalty= 'l2', multi_class='crammer_singer', random_state = 500)  
linSVC.fit(count_vect_train, y_train) 

predictions_linSVC = linSVC.predict(count_vect_test)
print(confusion_matrix(y_test,predictions_linSVC))  
print(classification_report(y_test,predictions_linSVC))  
print('Accuracy Score: ',accuracy_score(y_test, predictions_linSVC))
print('Train Accuracy Score: ',linSVC.score(count_vect_train,y_train))
print('Test Accuracy Score: ',linSVC.score(count_vect_test,y_test))

[[1331   72  164]
 [  41 3487   32]
 [ 123   55 3637]]
              precision    recall  f1-score   support

          -1       0.89      0.85      0.87      1567
           0       0.96      0.98      0.97      3560
           1       0.95      0.95      0.95      3815

    accuracy                           0.95      8942
   macro avg       0.93      0.93      0.93      8942
weighted avg       0.94      0.95      0.95      8942

Accuracy Score:  0.9455379109818832
Train Accuracy Score:  0.9970922917773366
Test Accuracy Score:  0.9455379109818832


/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:947: ConvergenceWarning:

Liblinear failed to converge, increase the number of iterations.



# **Multi-Class Logistic Regression**

In [ ]:
from sklearn.linear_model import LogisticRegression

logRes = LogisticRegression(C=1, random_state=0,max_iter=100,multi_class='multinomial')
logRes.fit(count_vect_train, y_train)


predictions_logRes = logRes.predict(count_vect_test)

print(confusion_matrix(y_test,predictions_logRes))  
print(classification_report(y_test,predictions_logRes))  
print('Accuracy Score: ',accuracy_score(y_test, predictions_logRes))
print('Train Accuracy Score: ',logRes.score(count_vect_train,y_train))
print('Test Accuracy Score: ',logRes.score(count_vect_test,y_test))


[[1268  123  176]
 [  28 3493   39]
 [  88   87 3640]]
              precision    recall  f1-score   support

          -1       0.92      0.81      0.86      1567
           0       0.94      0.98      0.96      3560
           1       0.94      0.95      0.95      3815

    accuracy                           0.94      8942
   macro avg       0.93      0.91      0.92      8942
weighted avg       0.94      0.94      0.94      8942

Accuracy Score:  0.9394989935137553
Train Accuracy Score:  0.9878659099169625
Test Accuracy Score:  0.9394989935137553


/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning:

lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression



# **Adaboost Classifier**

In [ ]:
from sklearn.ensemble import AdaBoostClassifier

abc = AdaBoostClassifier(n_estimators = 300, learning_rate = 1)
abc.fit(count_vect_train, y_train)
 
predictions_abc = abc.predict(count_vect_test)

print(confusion_matrix(y_test,predictions_abc))  
print(classification_report(y_test,predictions_abc))  
print('Accuracy Score: ',accuracy_score(y_test, predictions_abc))
print('Train Accuracy Score: ',abc.score(count_vect_train,y_train))
print('Test Accuracy Score: ',abc.score(count_vect_test,y_test))

[[1040  281  246]
 [  23 3505   32]
 [ 196  229 3390]]
              precision    recall  f1-score   support

          -1       0.83      0.66      0.74      1567
           0       0.87      0.98      0.93      3560
           1       0.92      0.89      0.91      3815

    accuracy                           0.89      8942
   macro avg       0.87      0.85      0.86      8942
weighted avg       0.89      0.89      0.88      8942

Accuracy Score:  0.8873853723999106
Train Accuracy Score:  0.8951267928537479
Test Accuracy Score:  0.8873853723999106
